# Lecture7 重回帰、statsmodelsにおけるカテゴリー変数の取り扱い

## 血液型の例から。

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# ダミー変数を作成
blood_types = ['A', 'B', 'AB', 'O']
data = {'blood_type': ['A', 'A', 'B', 'B', 'AB', 'AB', 'O', 'O'],
        'value': [1, 2, 2, 3, 3, 4, 4, 5]}
df = pd.DataFrame(data)
dummy_vars = pd.get_dummies(df['blood_type'])

In [ ]:
dummy_vars

In [ ]:
#ダミー変数dummy_varsをつかって回帰をします。
# 回帰モデルを作成
model = smf.ols('value ~ A + B + AB', data=dummy_vars.join(df)).fit()

model.summary()

In [ ]:
#可視化をします。
# 回帰係数の差分を取得
coeff_diff = model.params[1:] - model.params[0]

# 棒グラフを作成
plt.bar(blood_types[:-1], coeff_diff)
plt.xlabel('Blood Type')
plt.ylabel('Coefficient Difference')
plt.title('Coefficients Differences by Blood Type')
plt.show()

ご参考：例では、3変数を用いた回帰を行い、その係数から、効果の差分を見ました。４変数全部を用いると、多重共線性でうまく推定できない場合があります。

In [ ]:
model = smf.ols('value ~ A + B + AB + O', data=dummy_vars.join(df)).fit()

model.summary()

## 以下は別の例、
「Pythonで学ぶ計量経済学」より

https://py4etrics.github.io/13_Dummies.html

In [ ]:
!pip install wooldridge

In [ ]:
from statsmodels.formula.api import ols
import wooldridge

## ケース１：定数項だけの回帰分析
### ダミー変数なし
直感的な説明にするために説明変数が定数項だけの回帰分析から始める。 具体的には次の回帰式を考える。

$y=\beta_0 + \epsilon$

実は，この場合のOLS推定量
は被説明変数の平均と等しいことになる。この結果を確認するために以下ではwooldridgeパッケージのwage1のデータを使う。

In [ ]:
wage1 = wooldridge.data('wage1')
wooldridge.data('wage1',description=True)

In [ ]:
form_const = 'wage ~ 1'  # 定数項だけの場合は１が必要

res_const = ols(form_const, data=wage1).fit()
res_const.params

この$\beta_0$は、平均と同じ

In [ ]:
wage1['wage'].mean()

## ダミー変数あり：２つのケース
同じデータを使って$\{0,1\}$の値を取るダミー変数を考える。データセットwage1の中のfemaleという変数があり，以下の値を取る。

>女性の場合：female = 1</br>
>男性の場合：female = 0

値が０のカテゴリーを基本カテゴリーという。

$D$をfemaleのダミー変数とすると回帰式は以下のようになる。

$$y=\beta_0+\beta_1 D $$

さらに，この式はの$D=\{0,1\}$の値によって以下のように表すことができる。

男性：$D=0 \implies y=\beta_0 +\epsilon $

女性：$D=1 \implies y=\beta_0 +\beta_1 D+\epsilon $

即ち，OLS推定量は以下を表すことになる。

$\hat{\beta_0}$：男性の平均賃金

$\hat{\beta_0}+\hat{\beta_1}$：女性の平均賃金

この回帰式を使い，時間賃金の男女間の差について考察する

In [ ]:
form_const_2 = 'wage ~ female'

res_const_2 = ols(form_const_2, data=wage1).fit()

res_const_2.params

In [ ]:
res_const_2.params.Intercept + res_const_2.params.female

というわけで、女性の時間賃金は約2.51ドル低い

## ダミー変数あり：４つのケース
データセットwage1にはmarriedという変数が含まれており，以下の値をとる。

> 既婚者の場合：married = 1</br>
> 未婚者の場合：married = 0

femaleと組み合わせることにより，次の４つのケースを分けることができる。

>未婚男性：female=0, married=0</br>
>未婚女性：female=1, married=0</br>
>既婚女性：female=1, married=1</br>
>既婚男性：female=0, married=1

この４つのケースを捉えるために、femaleとmarriedの値によって0もしくは1の値になるダミー変数を作成するが，２つのステップに分けて説明する。

### カテゴリ変数を無視して回帰を行った場合

In [ ]:
form_const_3 = 'wage ~ female+married'

res_const_3 = ols(form_const_3, data=wage1).fit()

res_const_3.summary()

In [ ]:
print("singmale:",res_const_3.params.Intercept)
print("marmale:",res_const_3.params.Intercept+res_const_3.params.married)
print("singfem:",res_const_3.params.Intercept+res_const_3.params.female)
print("marfem:",res_const_3.params.Intercept+res_const_3.params.female+res_const_3.params.married)

In [ ]:
#答え合わせ独身男性の賃金平均
wage1.query('female==0 & married==0')['wage'].mean()

単純に重回帰しただけだと、答え合わせと合わない。

そこで、カテゴリー変数を導入する

$y= \beta_0+\beta_1 D_1 + \beta_2 D_2 + \beta_3 D_3$

- 基本カテゴリー：singmale
- $D_1$=marmale  
- $D_2$=singfem
- $D_3$=marfem

$D_1=\{0,1\},D_2=\{0,1\},D_3=\{0,1\}$の取る値を考慮すると，以下の４つのパターンに分けることができる。


$D_1=0 \& D_2=0 \& D_3=0 \implies y=\beta_0+\epsilon$

$D_1=1 \& D_2=0 \& D_3=0 \implies y=\beta_0+\beta_1 +\epsilon$

$D_1=0 \& D_2=1 \& D_3=0 \implies y=\beta_0+\beta_2+\epsilon$

$D_1=0 \& D_2=0 \& D_3=1 \implies y=\beta_0+\beta_3+\epsilon$


即ち，OLS推定量は以下を表すことになる。

$\hat{\beta_0}$：未婚男性の平均賃金

$\hat{\beta_0}+\hat{\beta_1}$：既婚男性の平均賃金

$\hat{\beta_0}+\hat{\beta_2}$：未婚女性の平均賃金

$\hat{\beta_0}+\hat{\beta_3}$：既婚女性の平均賃金

In [ ]:
# 以下では row をDataFrameの行と考える。

# 未婚男性の関数
def singmale(row):
    if row['female'] == 0 and row['married'] == 0:
        return 1
    else:
        return 0

# 既婚男性の関数
def marmale(row):
    if row['female'] == 0 and row['married'] == 1:
        return 1
    else:
        return 0

# 未婚女性の関数
def singfem(row):
    if row['female'] == 1 and row['married'] == 0:
        return 1
    else:
        return 0

# 既婚女性の関数
def marfem(row):
    if row['female'] == 1 and row['married'] == 1:
        return 1
    else:
        return 0

In [ ]:
wage1.loc[:,'singmale'] = wage1.apply(singmale, axis=1)  # axis='columns'でもOK
wage1.loc[:,'marmale'] = wage1.apply(marmale, axis=1)    # axis='columns'でもOK
wage1.loc[:,'singfem'] = wage1.apply(singfem, axis=1)    # axis='columns'でもOK
wage1.loc[:,'marfem'] = wage1.apply(marfem, axis=1)      # axis='columns'でもOK

wage1.head(3)

In [ ]:
form_const_4 = 'wage ~ marmale + singfem + marfem'

res_const_4 = ols(form_const_4, data=wage1).fit()

res_const_4.params

In [ ]:
#答え合わせ独身男性の賃金平均
wage1.query('female==0 & married==0')['wage'].mean()

In [ ]:
#答え合わせ既婚男性の賃金平均
wage1.query('female==0 & married==1')['wage'].mean()

In [ ]:
res_const_4.params.Intercept+res_const_4.params.marmale

## ケース２：定量的変数の導入
１つのダミー変数femaleだけが入るケースに次の変数を加えた回帰式を考える。

- educ：教育年数
- exper：雇用経験年数
- tenure：勤続年数

In [ ]:
form_1 = 'wage ~ female + educ + exper+ tenure'

res_1 = ols(form_1, data=wage1).fit()

res_1.params

賃金格差は約-1.81に減少した。これはeduc, exper, tenureの影響を取り除いた結果である。言い換えると，教育，経験，就労期間を所与とすると（それらの変数が同じである場合という意味），女性の時間賃金は約1.8ドル低い。

## ケース３：ダミー変数の交差項
ケース１と２の被説明変数はwageをそのまま使ったが，ここでは対数を取り賃金方程式にダミー変数の交差項を入れて考察する。

以下の回帰式を考える。

$$y=\beta_0+\beta_1 D+ \beta_2 Dx + \beta_3 x +\epsilon $$


ここで$D$がダミー変数，$x$は定量的変数であり，$Dx$がダミー変数の交差項である。ダミー変数が取る値$D=\{0,1\}$に分けて考えると，以
下を推定することになる。

$D=0 \implies y=\beta_0+\beta_3x+\epsilon$

$D=1 \implies y=(\beta_0+\beta_1)+(\beta_2+\beta_3)x+\epsilon$

具体例として $D$=female，$x$=educとするとOLS推定量は以下を表すことになる。

$\hat{\beta_0}$：（教育の効果を取り除いた）男性の平均賃金（対数）

$\hat{\beta_3}$：男性の賃金に対する教育の効果（％）

$\hat{\beta_0}+\hat{\beta_1}$：（教育の効果を取り除いた）女性の平均賃金（対数）

$\hat{\beta_2}+\hat{\beta_3}$：女性の賃金に対する教育の効果（％）


In [ ]:
import numpy as np
form_2 = 'np.log(wage) ~ female + female:educ + educ + exper + tenure'

res_2 = ols(form_2, data=wage1).fit()
res_2.summary().tables[1]

### t検定
- female
  - 教育などの影響を省いた後の平均賃金の差
  - 5%有意水準で$H_0$female=0は棄却できない。
- female:educ
  - 教育などの影響を省いた後の教育の収益率の差
  - 5%有意水準で$H_0$female:educ=0は棄却できない。

### F検定
$H_0$:female=female:educ=0の制約を考えよう

In [ ]:
hypotheses = 'female=0, female:educ=0'

res_2.f_test(hypotheses).pvalue

$H_0$は棄却される。

t検定では，femaleとfemale:educはそれぞれの帰無仮説が棄却されなかったが，F検定では制約が棄却された。一貫性がなく説明変数が不足している可能性がある。

# カテゴリー変数
カテゴリー変数とは定性的な変数であり，男女もカテゴリー変数の一種である。カテゴリー変数をダミー変数に変換するには2つの方法がある。

statsmodelsにはカテゴリー変数に自動的にダミー変数を割り当てる機能がある。操作は簡単で，単に回帰式の中でC()の中にカテゴリー変数を入れるだけである。

In [ ]:
df = wage1.loc[:,['wage', 'female', 'educ']]

dfのメソッドreplace()を使ってfemaleの列の値を以下のルールに沿って変換し，それをdfにsexという列として入れ直す。

- $1 \implies$  female
- $0 \implies$ male

In [ ]:
df.loc[:,'sex'] = df['female'].replace({1:'female',0:'male'})

In [ ]:
df.head(3)

In [ ]:
form_c = 'wage ~  C(sex) + educ'

res_c = ols(form_c, data=df).fit()

res_c.params

C(sex)[T.male]について

- TはTreatmentの頭文字で，通常のダミー変数を設定することを示している。
- maleはmaleの変数であることを表しており，自動的にfemaleが基本カテゴリーに設定されたことが分かる。

（結果）

C(sex)[T.male]はfemaleに比べてmaleの賃金は約2.27ドル高いことを示している。

## 蛇足
femaleとmarryedを独立に重回帰分析しても、答え合わせと合わなかったのは交絡因子をむししていたから。

In [ ]:
form_const_3 = 'wage ~ female*married'

res_const_3 = ols(form_const_3, data=wage1).fit()

res_const_3.summary()